In [ ]:
import datetime as dt
import numpy as np
import numba as nb
import pandas as pd
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from gas_storage.gas_price_simulations import GasPriceSimulations

##### Prices simulations

In [ ]:
simulator = GasPriceSimulations()
paths = simulator.get_simulations()
fig = go.Figure()
for path in paths[:50, :]:
    fig.add_trace(
        go.Scatter(
            x=simulator.index, y=path, showlegend=False, line=dict(color="lightgray")
        )
    )
fig = fig.update_layout(template="plotly_dark", title="Gas spot price paths")
# fig.show()

### Sets and parameters
$$
\begin{align*}
T &\dots \text{number of days} \\
N &\dots \text{number of simulations} \\
I_{max} &\dots \text{maximal inventory state} \\
q_{in},\;q_{out} &\dots \text{maximal injection and withdrawal rate} \\
\mathcal{I} &= \{0,\Delta I, 2\Delta I, \dots, I_{max}\} \\
t &\in \{0,1,\dots,T\} \\
k &\in \{0, 1,\dots,N\} \\
S^{(k)}_t\in\mathbb{R} &\dots \text{gas spot price simulation $k$ at time $t$} \\
a &\dots \text{action on the inventory} \\
\mathcal{A}(I) &= \{a\in\{q_{in},0,q_{out}\}\mid I+a\in\mathcal{I} \}\\
\end{align*}
$$

### Variables
Inventory state (realized) at time $t$
$$
\begin{align*}
I_t &\in \mathcal{I} \\
\end{align*}
$$

Decision variable
$$
\begin{align*}
a_t &\in \mathcal{A}(I_t)
\end{align*}
$$

##### Value function
$$
\begin{align*}
V_t(I) &= \sup_{\{a_{\tau}\}_{\tau=t}^{T-1}}\mathbb{E}[\sum^{T-1}_{\tau=t}(-a_{\tau}S_{\tau})+I_T S_T\mid I_t=I] \\
V_t(I) &= \sup_{\{a_{\tau}\}_{\tau=t}^{T-1}}\mathbb{E}[\sum^{T-1}_{\tau=t}(-a_{\tau}S_{\tau})+(I+\sum^{T-1}_{\tau=t}a_{\tau})S_T] \\
\end{align*}
$$

##### Bellman equation
$$
V_t(I) = \max_{a\in \mathcal{A}(I)}\{-a S_t + \mathbb{E}[V_{t+1}(I+a)\mid S_t]\} \\
$$
Value today $=$ immediate cashflow from the best chosen action $+$ expected value of tomorrow. <br>
This expected value depends on today's price - the next day action $a_t$ depends on the price and inventory.

##### Constraints
$$
\begin{align*}
I_{t+1} &= I_t + a_t \\
V_T(I) &= I\cdot S_T,\quad \forall I\in\mathcal{I} \\
\end{align*}
$$

##### Expectation handling
$$
\mathbb{E}[V_{t+1}(I_{t+1})\mid S_t,I_t]
$$
What is the average future value of the storage tomorrow, given what I know today: today's price $S_t$ and today's inventory $I_t$.

##### Summary
At each time $t$ we compute the value of each possible inventory state using many simulated prices to estimate the expected future value.

### Example
$$
\begin{align*}
t &\dots \text{today} \\
S_t &= 30\; \text{EUR/MWh} \\
I_t &= 50\; \text{MWh} \\
a_t &\in \{-5, 0, 5\} \\
V_t(I_t) &= \text{Maximum expected future profit from today onward} \\
\end{align*}
$$

##### Action 1 - inject ($a_t = 5$)
$$
\begin{align*}
I_{t+1} &= 50+5=55 \\
-a_t\cdot S_t &= -5\cdot 30 = -150 \\
V_{t+1}(55) &= \text{?} \\
\text{value} &: −150+\mathbb{E}[V_{t+1}​(55) \mid S_t​=30,I_t​=50]
\end{align*}
$$

##### Action 2 - hold ($a_t = 0$)
$$
\begin{align*}
I_{t+1} &= 50 \\
-a_t\cdot S_t &= 0 \\
\text{value} &: \mathbb{E}[V_{t+1}​(50) \mid S_t​=30,I_t​=50]
\end{align*}
$$

##### Action 3 - withdraw ($a_t = -5$)
$$
\begin{align*}
I_{t+1} &= 45 \\
-a_t\cdot S_t &= -(-5)\cdot 30 = 150 \\
\text{value} &: 150+\mathbb{E}[V_{t+1}​(45) \mid S_t​=30,I_t​=50]
\end{align*}
$$

##### Decision
$$
\begin{align*}
\text{inject} &: −150+\mathbb{E}[V_{t+1}​(55) \mid S_t​=30,I_t​=50] \\
\text{hold} &: \mathbb{E}[V_{t+1}​(50) \mid S_t​=30,I_t​=50] \\
\text{withdraw} &: 150+\mathbb{E}[V_{t+1}​(45) \mid S_t​=30,I_t​=50] \\
\end{align*}
$$

In [ ]:
I_max = 100.0
q_in = 5.0
q_out = 5.0

n_I = 51
I_grid = np.linspace(0.0, I_max, n_I)


n_paths, n_steps = paths.shape

V = np.zeros((n_paths, n_I))
S_T = paths[:, -1]

for i, I in enumerate(I_grid):
    V[:, i] = I * S_T

actions = np.array([-q_out, 0.0, q_in])

In [ ]:
def continuation_value(S, Y):
    # regress Y on [1, S, S^2]
    X = np.column_stack([np.ones_like(S), S, S**2])
    beta, _, _, _ = np.linalg.lstsq(X, Y, rcond=None)
    return X @ beta

In [ ]:
for t in reversed(range(n_steps - 1)):
    print(t)
    S_t = paths[:, t]
    V_next = V.copy()

    for i, I in enumerate(I_grid):
        values = []

        for a in actions:
            I_new = I + a
            if I_new < 0 or I_new > I_max:
                values.append(np.full_like(S_t, -np.inf))
                continue

            j = np.argmin(np.abs(I_grid - I_new))

            cont = continuation_value(S_t, V_next[:, j])
            payoff = -a * S_t + cont
            values.append(payoff)

        V[:, i] = np.max(np.column_stack(values), axis=1)

In [ ]:
V.shape

In [ ]:
I0 = 50.0
i0 = np.argmin(np.abs(I_grid - I0))

storage_value = V[:, i0].mean()
print(f"Estimated storage value: {storage_value:.2f}")